# Downloading comments from youtube
In this section we will try different methods to download comments from youtube videos.

First we will try with conventional methods with requests and parsing the results with Beautiful Soup

In [3]:
import requests
import json
from bs4 import BeautifulSoup

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'

def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]

def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)

def get_comments(youtube_id):
    youtube_comments_url = "https://www.youtube.com/all_comments?v=%s" % youtube_id
    youtube_comments_ajax_url = "https://www.youtube.com/comment_ajax"
    

In [4]:
youtube_id = "1Y-Au-tnBLs"
youtube_comments_url = "https://www.youtube.com/all_comments?v=%s" % youtube_id
youtube_comments_ajax_url = "https://www.youtube.com/comment_ajax"

session = requests.Session()
session.headers['User-Agent'] = USER_AGENT
response = session.get(youtube_comments_url)
soup = BeautifulSoup(response.content)

page_token = find_value(response.text, 'data-token')
xsrf_token = find_value(response.text, "XSRF_TOKEN", 4)

data = {'video_id': youtube_id,
                'session_token': xsrf_token}

params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

first_iteration = True
if first_iteration:
    params['order_menu'] = True
else:
    data['page_token'] = page_token

response = ajax_request(session, youtube_comments_ajax_url, params, data)
print(response)
#print(page_token)
#print(xsrf_token)
#print(soup.prettify())

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


('', '  \n\n      <div class="comment-entry">\n        \n\n\n\n\n\n\n\n\n  <div class="comment-item yt-commentbox-top" data-aid="UCVQrCq0eIS6OoYnJraGJVnw" data-cid="zXUFGiMTkrTUZommMK5zcQ5CJN_GkVVa5JIf_YRo7AI" data-vid="1Y-Au-tnBLs" data-name="smashingbear">\n\n\n    <a href="/user/smashingbear" target="_blank" class="g-hovercard" data-ytid="UCVQrCq0eIS6OoYnJraGJVnw">\n      <img class="user-photo" src="https://gp6.googleusercontent.com/-HDnFZ7YQYtc/AAAAAAAAAAI/AAAAAAAAAAA/GHpicuwqoG8/s48-c-k-no/photo.jpg?sz=50" width="48">\n    </a>\n    <div class="content">\n        <div class="comment-header">\n    <a href="/user/smashingbear" class="user-name g-hovercard" target="_blank" data-ytid="UCVQrCq0eIS6OoYnJraGJVnw">smashingbear</a>\n    <span class="spacer"></span>\n    <span class="time">\n        <a href="/watch?v=1Y-Au-tnBLs&amp;lc=zXUFGiMTkrTUZommMK5zcQ5CJN_GkVVa5JIf_YRo7AI" target="_blank">\n          5 years ago\n        </a>\n    </span>\n  </div>\n\n        <div class="comment-tex

In [5]:
import os
import sys
import time
import json
import requests
import argparse
import lxml.html

from lxml.cssselect import CSSSelector

YOUTUBE_COMMENTS_URL = 'https://www.youtube.com/all_comments?v={youtube_id}'
YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'


def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]


def extract_comments(html):
    tree = ""
    try:
        tree = lxml.html.fromstring(html)
    except Exception:
        return {'cid': 'XXXX',
               'text': 'XXXX',
               'time': 'XXXX',
               'author': 'XXXX'}
        pass
        # Log error
    item_sel = CSSSelector('.comment-item')
    text_sel = CSSSelector('.comment-text-content')
    time_sel = CSSSelector('.time')
    author_sel = CSSSelector('.user-name')

    for item in item_sel(tree):
        yield {'cid': item.get('data-cid'),
               'text': text_sel(item)[0].text_content(),
               'time': time_sel(item)[0].text_content().strip(),
               'author': author_sel(item)[0].text_content()}


def extract_reply_cids(html):
    tree = ""
    try:
        tree = lxml.html.fromstring(html)
    except Exception as err:
        raise
        # TODO: log these errors
        # print("Error: {0} ".format(err) + " while opening: " + html) 
    sel = CSSSelector('.comment-replies-header > .load-comments')
    return [i.get('data-cid') for i in sel(tree)]


def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)

def is_video_private(youtube_id):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT

    # Get Youtube page with initial comments
    response = session.get(YOUTUBE_COMMENTS_URL.format(youtube_id=youtube_id))
    html = response.text
    # Check if video is private
    return "video is private" in html
        

def download_comments(youtube_id, sleep=1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT

    # Get Youtube page with initial comments
    response = session.get(YOUTUBE_COMMENTS_URL.format(youtube_id=youtube_id))
    html = response.text
    # Check if video is private
    if "video is private" in html:
        return {'cid': 'XXXX',
               'text': 'XXXX',
               'time': 'XXXX',
               'author': 'XXXX'}
        
    reply_cids = 'XXXX'
    try:
        reply_cids = extract_reply_cids(html)
    except:
        pass

    ret_cids = []
    for comment in extract_comments(html):
        ret_cids.append(comment['cid'])
        yield comment

    page_token = find_value(html, 'data-token')
    session_token = find_value(html, 'XSRF_TOKEN', 4)

    first_iteration = True

    # Get remaining comments (the same as pressing the 'Show more' button)
    while page_token:
        data = {'video_id': youtube_id,
                'session_token': session_token}

        params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

        if first_iteration:
            params['order_menu'] = True
        else:
            data['page_token'] = page_token

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        page_token, html = response

        try:
            reply_cids += extract_reply_cids(html)
        except:
            reply_cids += 'XXXX'
            pass
        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment

        first_iteration = False
        time.sleep(sleep)

    # Get replies (the same as pressing the 'View all X replies' link)
    for cid in reply_cids:
        data = {'comment_id': cid,
                'video_id': youtube_id,
                'can_reply': 1,
                'session_token': session_token}

        params = {'action_load_replies': 1,
                  'order_by_time': True,
                  'filter': youtube_id,
                  'tab': 'inbox'}

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        _, html = response

        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment
        time.sleep(sleep)

In [6]:
%%time
is_video_private("7zCIRPQ8qWc")

CPU times: user 32.2 ms, sys: 4.53 ms, total: 36.7 ms
Wall time: 610 ms


False

In [7]:
%%time
is_video_private("fY_FQMQpjok")

CPU times: user 31.9 ms, sys: 3.62 ms, total: 35.5 ms
Wall time: 372 ms


True

In [4]:
%%time
youtube_id = "7zCIRPQ8qWc"
for comment in download_comments(youtube_id):
    print(comment['text'])

"Hilarious" isn't the word I would use. I would use "boring".﻿
great video !!!!!!!!!!﻿
that's so funny LLLLLLLOOOOOOOOLLLLLLLL!!!!!!!!﻿
Waha haha hahaha﻿
Waha haha hahaha﻿
i couldn't stop laughing and feeling bad for the cats
﻿
ik huilde bijna van het lachen﻿
OMG XD﻿
Суууууууууууууууупер﻿
We were ALL kids once! Adults are even suckier!﻿
True, but nows the time to teach him/her NOT to do that kinda stuff to kitty! :)
They learn quick!﻿
I agree! I saw another video like this and the 3 year old kid threw a cat into the pool! I would've whacked the parents upside the head and swatted the kid on the butt and tell him to NEVER ever be mean to any animal, especially throwing our kitty into water! Yeah, that "baby" looked like he was trying to drown the cat! The parents better watch out for that, it can lead to other things with animals that are hurtful to them! And he could have gotten badly scratched up too.﻿
Epic - Cats the song xD
/watch?v=Nxxu0i9UVMk﻿
The last two aren't really funny. the

ConnectionError: ('Connection aborted.', OSError(65, 'No route to host'))

In [34]:
# Now check private video
youtube_id = "fY_FQMQpjok"
for comment in download_comments(youtube_id):
    print(comment['text'])

# Reading the dataset

In [24]:
def unique_videos_in_dataset(file):
    with open(file) as csvfile:
        encountered = {}
        videos = []
        youtube_id = csv.reader(csvfile, delimiter = ',')
        unique = 0
        for row in youtube_id:
            if row[0] not in encountered:
                encountered[row[0]] = True
                videos += [row[0]]
                unique += 1
            if row[1] not in encountered:
                encountered[row[1]] = True
                videos += [row[1]]
                unique += 1
        return videos

train_unique = unique_videos_in_dataset('comedy_comparisons.train')
test_unique = unique_videos_in_dataset('comedy_comparisons.test')
with open('train_unique', 'w') as t:
    t.write("\n".join(train_unique))
with open('test_unique', 'w') as t:
    t.write("\n".join(test_unique))

print("Found {0} train unique videos and {1} test videos.".format(len(train_unique), len(test_unique)))

Found 18474 train unique videos and 4263 test videos.


In [23]:
"\n".join(train_unique)

'sNabaB-eb3Y\nwHkPb68dxEw\ny2emSXSE-N4\nfY_FQMQpjok\nVr4D8xO2lBY\ndDtRnstrefE\nwSMh4E3xxdw\nNo-ZR7-X76s\nHZPUQQNRvOg\n2ruyMxxfAJg\nvX95JgKGu0o\nwu4SU70w7LA\n2FHH-9teZP0\nsP3YdfG_Uio\n-AZsrDqF83A\nDqEJS86_iUA\nSG2B7dIqbNQ\nrhjIStU0JvI\naxZ2xvYhOKw\na-syzH14WQA\nTgrUs_ZwwDc\nbfU4rTa-PfQ\njpntOJ_xDJ0\nLr4SS1zxRYg\nQwpWaLrDHnU\nwvybYNEDQWE\nAi6DSPPo0VI\ng6lJ2k3TCFg\nUPTrASEx_p8\ng2HylVyOyzc\nplieAqK2a00\nJmtITSojOF8\nBXPvq89jMUg\nkRtbOTXN0FM\nsfQEd-lVHCE\nwY-1KNh7-NQ\naAjvqy_L3C8\nYWzqfaX4ofU\not0YhG58PiM\n-0lrQdFmL9U\ndRZIWb_Nzwo\n4gDfq2vxyXk\nyqEeP1acj4Y\nbYwTrq__zV4\nKPP5NbjW9eI\npa22SdOuWnk\njJ3eeDp4tkQ\nHIwA2P7aeTk\nA5ECnsNtg1c\nUomk60wP2mI\nXMK89HXKKAA\ncvoOHcAlY2o\nElL6I6-jjF4\n3-i0l0EKeqc\nNlYMZ-bWWnE\n39NV1EZvVgk\ndbEq8b3y8ws\n1yLgfOXYg_I\nnXR6l70Pmqk\n2HFTEykZ8mY\nQ6MKYbWPX5Y\nije7uAIuO1g\nqtXFbTTW8es\nGGuqLRDmfhw\npsO6_rNKP6g\nvA63Ln5xbDk\nap9Fp8lBtMo\nDdSUjqtYBAc\neDz1Oi8yR3w\nqrrAB-wu_8s\nihTJHXm93v0\nw8UWgufUIv0\nl1Sj3HSXGxw\ncUptk3m1Ro4\nwHOraxZ3RKM\nNA3I4aZk0fQ\nVESnbnjgD4s

In [27]:
# Counting the number of unique and non private videos in the dataset
from tqdm import tqdm, tqdm_notebook, tnrange
from time import sleep

def non_private_videos(idlist):
    videos = []
    pbar = tqdm_notebook(range(len(idlist)), desc="Downloading video")
    for video_id in idlist:
        if not is_video_private(video_id):
            videos += [video_id]
        pbar.update(1)
        sleep(0.5)
    return videos
      
train = non_private_videos(train_unique)
test = non_private_videos(test_unique)
with open('train_unique_non_private', 'w') as t:
    t.write("\n".join(train))
with open('test_unique_non_private', 'w') as t:
    t.write("\n".join(test))
print("There are {0} unique non-private videos in train and {1} unique non-private in test".format(len(train), len(test)))

There are 17534 unique non-private videos in train and 4055 unique non-private in test


['sNabaB-eb3Y',
 'wHkPb68dxEw',
 'y2emSXSE-N4',
 'Vr4D8xO2lBY',
 'dDtRnstrefE',
 'wSMh4E3xxdw',
 'HZPUQQNRvOg',
 '2ruyMxxfAJg',
 'vX95JgKGu0o',
 'wu4SU70w7LA',
 '2FHH-9teZP0',
 'sP3YdfG_Uio',
 'SG2B7dIqbNQ',
 'rhjIStU0JvI',
 'axZ2xvYhOKw',
 'TgrUs_ZwwDc',
 'bfU4rTa-PfQ',
 'jpntOJ_xDJ0',
 'Lr4SS1zxRYg',
 'QwpWaLrDHnU',
 'wvybYNEDQWE',
 'g6lJ2k3TCFg',
 'UPTrASEx_p8',
 'g2HylVyOyzc',
 'plieAqK2a00',
 'JmtITSojOF8',
 'BXPvq89jMUg']

In [18]:
# Select 30 videos at random to then measure the average time to download
import numpy as np
selected = []
file = './comedy_comparisons.train'
with open(file) as csvfile:
    encountered = {}
    youtube_id = csv.reader(csvfile, delimiter = ',')
    for row in youtube_id:
        if row[0] not in encountered:
            if np.random.random_sample() < 0.5:
                selected += [row[0]]
                if len(selected) >= 30:
                    break
            encountered[row[0]] = True
            unique += 1
        if row[1] not in encountered:
            if np.random.random_sample() < 0.5:
                selected += [row[1]]
                if len(selected) >= 30:
                    break
            encountered[row[1]] = True
            unique += 1

selected



['sNabaB-eb3Y',
 'fY_FQMQpjok',
 'No-ZR7-X76s',
 'HZPUQQNRvOg',
 '2ruyMxxfAJg',
 'vX95JgKGu0o',
 'wu4SU70w7LA',
 '-AZsrDqF83A',
 'rhjIStU0JvI',
 'axZ2xvYhOKw',
 'TgrUs_ZwwDc',
 'jpntOJ_xDJ0',
 'Lr4SS1zxRYg',
 'QwpWaLrDHnU',
 'Ai6DSPPo0VI',
 'g6lJ2k3TCFg',
 'UPTrASEx_p8',
 'g2HylVyOyzc',
 'JmtITSojOF8',
 'BXPvq89jMUg',
 'YWzqfaX4ofU',
 'ot0YhG58PiM',
 '-0lrQdFmL9U',
 'dRZIWb_Nzwo',
 'bYwTrq__zV4',
 'pa22SdOuWnk',
 'jJ3eeDp4tkQ',
 'HIwA2P7aeTk',
 'Uomk60wP2mI',
 'NlYMZ-bWWnE']

In [ ]:
%%time
import csv
import sys
import numpy as np
#with open('./comedy_comparisons.train') as csvfile:
#    youtube_id_reader = csv.reader(csvfile, delimiter = ',')
#visited = {}
i = 0
fieldnames = ['youtube_id', 'cid', 'text', 'time', 'author']
with open('dataset_movies.csv', 'a') as outputfile:
    writer = csv.DictWriter(outputfile, fieldnames=fieldnames)
    writer.writeheader()   
    for row in selected:
        #if np.random.random_sample() < 0.5:
        #    continue
        #if i > 29: break
       # youtube_id1 = row[0]
       # youtube_id2 = row[1]
        #print(row)
        def download_video_comments(youtube_id):  
            global i
            global visited
            #if youtube_id not in visited:
            for comment in download_comments(youtube_id):
                writer.writerow({'youtube_id': youtube_id, 'cid': comment['cid'], 'text': comment['text'], 'time': comment['time'], 'author': comment['author']})
                #try:
                #    print(str(youtube_id) + ':' + comment['text'])
                #    sys.stdout.flush()
                #except:
                #    pass

                #print(youtube_id)
                #visited[youtube_id] = True
               # i = i + 1
                #print(str(i) + "-" * 15 + "\n\n")

        download_video_comments(row)
       # download_video_comments(youtube_id2)


            

# Notes about progress
* All comments of all videos tested are downloaded successfully and can be saved in a CSV file
* However, many of the videos of the dataset are now private and must be cleaned
* Any rows containing private videos must be eliminated and then estimate the average time to download video comments (this code is already tested), to decide how many videos to use in the datasets train and test
* Finally create the final script to download the dataset

In [10]:
from time import sleep
from tqdm import tqdm, tqdm_notebook, tnrange

for i in tnrange(1000, desc="test loop"):
    sleep(0.01)